<a href="https://colab.research.google.com/github/Abhinaya-09/ML-LAB-EXPERIMENTS/blob/main/ML_LAB_EXPERIMENT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2 .For a given set of training data examples stored in a .CSV file implement and demonstrate the Candidate-Elimination algorithm to output a description of the set of all hypotheses consistent with the training examples.

In [1]:
import csv

def g_0(n):
    return ("?",) * n

def s_0(n):
    return ('0',) * n

def more_general(h1, h2):
    more_general_parts = []
    for x, y in zip(h1, h2):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
    return all(more_general_parts)


In [2]:
def fulfills(example, hypothesis):
    return more_general(hypothesis, example)

def min_generalizations(h, x):
    h_new = list(h)
    for i in range(len(h)):
        if not fulfills(x[i:i+1], h[i:i+1]):
            h_new[i] = '?' if h[i] != '0' else x[i]
    return [tuple(h_new)]

In [3]:
def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
        elif h[i] != "0":
            h_new = h[:i] + ('0',) + h[i+1:]
            results.append(h_new)
    return results

In [4]:
def get_domains(examples):
    d = [set() for i in examples[0]]
    for x in examples:
        for i, xi in enumerate(x):
            d[i].add(xi)
    return [list(sorted(x)) for x in d]


In [5]:


def candidate_elimination(examples):
    domains = get_domains(examples)[:-1]
    G = set([g_0(len(domains))])
    S = set([s_0(len(domains))])
    i = 0
    print("\n G[{0}]:".format(i), G)
    print("\n S[{0}]:".format(i), S)
    for xcx in examples:
        i = i + 1
        x, cx = xcx[:-1], xcx[-1]  # Splitting data into attributes and decisions
        if cx == 'Y':  # x is positive example
            G = {g for g in G if fulfills(x, g)}
            S = generalize_S(x, G, S)
        else:  # x is negative example
            S = {s for s in S if not fulfills(x, s)}
            G = specialize_G(x, domains, G, S)
        print("\n G[{0}]:".format(i), G)
        print("\n S[{0}]:".format(i), S)
    print("\nFinal General Hypothesis (G):", G)
    print("\nFinal Specific Hypothesis (S):", S)
    return

In [ ]:
def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not fulfills(x, s):
            S.remove(s)
            Splus = min_generalizations(s, x)
            # Keep only generalizations that have a counterpart in G
            S.update([h for h in Splus if any([more_general(g, h) for g in G])])
            # Remove hypotheses less specific than any other in S
            S.difference_update([h for h in S if any([more_general(h, h1) for h1 in S if h != h1])])
    return S

In [6]:
def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
        if fulfills(x, g):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            # Keep only specializations that have a counterpart in S
            G.update([h for h in Gminus if any([more_general(h, s) for s in S])])
            # Remove hypotheses less general than any other in G
            G.difference_update([h for h in G if any([more_general(g1, h) for g1 in G if h != g1])])
    return G

In [7]:

# Load the examples from a CSV file
with open('/content/Lab2.csv') as csvFile:
    examples = [tuple(line) for line in csv.reader(csvFile)]

In [11]:
# Function to check if a hypothesis fulfills an example
def fulfills(example, hypothesis):
    return all(h == '?' or h == e for h, e in zip(hypothesis, example))

# Function to generalize the specific hypothesis S
def generalize_S(example, G, S):
    new_S = list(S.copy())
    for s in S:
        if not fulfills(example, s):
            new_S.remove(s)
            general_hypothesis = list(s)
            for i in range(len(s)):
                if s[i] != example[i]:
                    general_hypothesis[i] = '?'
            if any(fulfills(general_hypothesis, g) for g in G):
                new_S.append(tuple(general_hypothesis))
    return set(new_S)

# Candidate elimination algorithm
def candidate_elimination(examples):
    G = {('?', '?', '?', '?', '?', '?')}
    S = {('0', '0', '0', '0', '0', '0')}

    for i, example in enumerate(examples):
        x, cx = example[:-1], example[-1]
        print(f"\nExample {i+1}: {example}")

        if cx == 'Y':  # Positive example
            G = {g for g in G if fulfills(x, g)}
            S = generalize_S(x, G, S)
        else:  # Negative example
            S = {s for s in S if not fulfills(x, s)}
            G = specialize_G(x, G, S)

        print(f"G[{i+1}]: {G}")
        print(f"S[{i+1}]: {S}")

# Function to specialize the general hypothesis G
def specialize_G(example, G, S):
    new_G = list(G.copy())
    for g in G:
        if fulfills(example, g):
            new_G.remove(g)
            for i in range(len(g)):
                if g[i] == '?':
                    specialized_hypothesis = list(g)
                    specialized_hypothesis[i] = example[i]
                    if any(fulfills(specialized_hypothesis, s) for s in S):
                        new_G.append(tuple(specialized_hypothesis))
    return set(new_G)

# Load the examples from a CSV file
import csv
with open('/content/Lab2.csv') as csvFile:
    examples = [tuple(line) for line in csv.reader(csvFile)]

# Run the candidate elimination algorithm
candidate_elimination(examples)



Example 1: ('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same', 'Y')
G[1]: {('?', '?', '?', '?', '?', '?')}
S[1]: {('?', '?', '?', '?', '?', '?')}

Example 2: ('Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same', 'Y')
G[2]: {('?', '?', '?', '?', '?', '?')}
S[2]: {('?', '?', '?', '?', '?', '?')}

Example 3: ('Rainy', 'Cold', 'High', 'Strong', 'Warm', 'Change', 'N')
G[3]: set()
S[3]: set()

Example 4: ('Sunny', 'Warm', 'High', 'Strong', 'Cool', 'Change', 'Y')
G[4]: set()
S[4]: set()
